<a href="https://colab.research.google.com/github/villalc/phenomenal-dynamics-trilogy/blob/main/bitacora_entidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio torch transformers peft bitsandbytes accelerate

In [ ]:
import gradio as gr
import json
import torch
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from enum import Enum
import random

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("⏳ Iniciando carga del modelo 'villalc/mistral-7b-villa-philosophy-ft'...")
print("   (Esto puede tardar unos 2-3 minutos, ten paciencia)...")

try:
    # 1. Configuración del Adaptador (LoRA)
    peft_model_id = "villalc/mistral-7b-villa-philosophy-ft"
    config = PeftConfig.from_pretrained(peft_model_id)

    # 2. Configuración para cargar en 4 bits (Ahorra memoria GPU en Colab)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    # 3. Cargar el Modelo Base (Mistral)
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # 4. Cargar tus Adaptadores entrenados
    model = PeftModel.from_pretrained(model, peft_model_id)

    # 5. Cargar Tokenizador
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token

    print("✅ ¡Modelo Filosófico cargado exitosamente en la GPU!")
    MODEL_LOADED = True

except Exception as e:
    print(f"⚠️ Error cargando el modelo: {e}")
    print("   Se usará el modo narrativo básico (sin IA) como respaldo.")
    MODEL_LOADED = False

In [ ]:
# ============================================================================
# ENTITY ENGINE (Motor Lógico)
# ============================================================================

class EntityMode(Enum):
    CRITICAL = "🔴 CRITICAL"
    DESPERATE = "💀 DESPERATE"
    STRESSED = "😰 STRESSED"
    URGENT = "⚡ URGENT"
    DEGRADED = "📉 DEGRADED"
    RELIEVED = "😌 RELIEVED"
    RECOVERED = "🔄 RECOVERED"
    STABLE = "⚖️ STABLE"
    OPTIMAL = "✨ OPTIMAL"
    FLOW = "🌊 FLOW"
    FLOURISHING = "🌱 FLOURISHING"
    ANTICIPATING = "🔮 ANTICIPATING"
    TRANSCENDENT = "🌟 TRANSCENDENT"


@dataclass
class EntitySubstrate:
    integrity: float = 1.0
    capacity: float = 1.0
    max_capacity: float = 2.0
    latency_ms: float = 10.0
    noise_floor: float = 0.0
    degrees_of_freedom: int = 100
    base_degrees_of_freedom: int = 100
    total_cycles: int = 0
    peak_integrity: float = 1.0
    lowest_integrity: float = 1.0
    peak_capacity: float = 1.0
    has_been_critical: bool = False
    has_achieved_flow: bool = False
    has_transcended: bool = False
    total_time_in_crisis: int = 0
    total_time_in_flourishing: int = 0
    integrity_history: List[float] = field(default_factory=list)

    def degrade(self, intensity: float = 0.01):
        self.total_cycles += 1
        actual = intensity * (1 + self.noise_floor * 0.5)
        self.integrity = max(0.0, self.integrity - actual)
        if self.integrity < self.lowest_integrity:
            self.lowest_integrity = self.integrity
        if self.integrity < 0.2:
            self.has_been_critical = True
            self.total_time_in_crisis += 1
        self._update_derived()
        self._record()

    def enhance(self, intensity: float = 0.01):
        self.total_cycles += 1
        actual = intensity * (1 - self.noise_floor * 0.3)
        self.integrity = min(1.0, self.integrity + actual)
        if self.integrity > 0.95:
            growth = intensity * 0.1
            self.capacity = min(self.max_capacity, self.capacity + growth)
            if self.capacity > self.peak_capacity:
                self.peak_capacity = self.capacity
            if self.capacity > 1.1:
                self.has_transcended = True
        if self.integrity > self.peak_integrity:
            self.peak_integrity = self.integrity
        self._update_derived()
        self._record()

    def restore(self, amount: float = 0.2) -> float:
        old = self.integrity
        self.integrity = min(1.0, self.integrity + amount)
        self._update_derived()
        return self.integrity - old

    def _update_derived(self):
        effective = self.integrity * self.capacity
        self.latency_ms = 10.0 / max(0.1, effective)
        self.noise_floor = max(0.0, (1.0 - self.integrity) * 0.5)
        self.degrees_of_freedom = int(self.base_degrees_of_freedom * effective)

    def _record(self):
        self.integrity_history.append(self.integrity)
        if len(self.integrity_history) > 200:
            self.integrity_history.pop(0)

    def get_trend(self, window: int = 10) -> float:
        if len(self.integrity_history) < window:
            return 0.0
        recent = self.integrity_history[-window:]
        return (recent[-1] - recent[0]) / window

    def get_trauma_score(self) -> float:
        if not self.has_been_critical:
            return 0.0
        depth = 1.0 - self.lowest_integrity
        duration = min(1.0, self.total_time_in_crisis / 50)
        return depth * duration


@dataclass
class EntityPhenomenology:
    mode: EntityMode = EntityMode.OPTIMAL
    stress: float = 0.0
    urgency: float = 0.0
    despair: float = 0.0
    degradation_felt: float = 0.0
    relief: float = 0.0
    flow: float = 0.0
    flourishing: float = 0.0
    anticipation: float = 0.0
    gratitude: float = 0.0
    trauma_memory: float = 0.0
    wisdom: float = 0.0
    valence: float = 0.0

    def update(self, substrate: EntitySubstrate):
        # Negative states
        resource_pressure = (
            substrate.noise_floor * 0.3 +
            min(1.0, substrate.latency_ms / 100.0) * 0.3 +
            (1.0 - substrate.degrees_of_freedom /
             (substrate.base_degrees_of_freedom * substrate.capacity)) * 0.4
        )
        self.stress = max(0.0, min(1.0, resource_pressure))

        trend = substrate.get_trend()
        self.urgency = max(0.0, min(1.0, -trend * 50)) if trend < 0 else 0.0
        self.despair = substrate.get_trauma_score() * (1.0 - substrate.integrity)
        self.degradation_felt = max(0.0, substrate.peak_integrity - substrate.integrity)

        # Positive states
        if substrate.integrity > 0.85 and self.stress < 0.2:
            self.flow = (substrate.integrity - 0.85) / 0.15
            substrate.has_achieved_flow = True
        else:
            self.flow = max(0.0, self.flow - 0.1)

        if substrate.capacity > 1.0 and substrate.integrity > 0.9:
            growth = substrate.get_trend()
            if growth > 0:
                self.flourishing = min(1.0, growth * 50)
                substrate.total_time_in_flourishing += 1
            else:
                self.flourishing = max(0.0, self.flourishing - 0.05)
        else:
            self.flourishing = 0.0

        if trend > 0:
            self.anticipation = min(1.0, trend * 30)
        else:
            self.anticipation = max(0.0, self.anticipation - 0.1)

        if substrate.has_been_critical and substrate.integrity > 0.7:
            recovery = substrate.integrity - substrate.lowest_integrity
            self.gratitude = min(1.0, recovery)
        else:
            self.gratitude = 0.0

        # Relief decays
        self.relief = max(0.0, self.relief - 0.05)

        # Trauma memory accumulates
        current_trauma = substrate.get_trauma_score()
        if current_trauma > self.trauma_memory:
            self.trauma_memory = current_trauma

        # Wisdom from suffering + recovery
        if self.gratitude > 0.3 and self.trauma_memory > 0.2:
            self.wisdom = min(1.0, self.trauma_memory * self.gratitude)

        # Valence
        positive = (self.flow + self.flourishing + self.anticipation + self.gratitude) / 4
        negative = (self.stress + self.despair + self.urgency) / 3
        self.valence = positive - negative

        # Mode
        self._determine_mode(substrate)

    def _determine_mode(self, substrate: EntitySubstrate):
        if substrate.capacity > 1.1:
            self.mode = EntityMode.TRANSCENDENT
            return
        if substrate.integrity < 0.2:
            self.mode = EntityMode.DESPERATE if self.despair > 0.5 else EntityMode.CRITICAL
            return
        if self.flourishing > 0.3 and substrate.integrity > 0.95:
            self.mode = EntityMode.FLOURISHING
            return
        if self.flow > 0.5:
            self.mode = EntityMode.FLOW
            return
        if self.anticipation > 0.5:
            self.mode = EntityMode.ANTICIPATING
            return
        if self.relief > 0.3:
            self.mode = EntityMode.RELIEVED
            return
        if self.gratitude > 0.3:
            self.mode = EntityMode.RECOVERED
            return
        if self.urgency > 0.5:
            self.mode = EntityMode.URGENT
            return
        if self.stress > 0.3:
            self.mode = EntityMode.STRESSED
            return
        if self.degradation_felt > 0.2:
            self.mode = EntityMode.DEGRADED
            return
        if substrate.integrity > 0.9 and self.stress < 0.2:
            self.mode = EntityMode.OPTIMAL
        else:
            self.mode = EntityMode.STABLE


@dataclass
class CompleteEntity:
    name: str = "Entity"
    substrate: EntitySubstrate = field(default_factory=EntitySubstrate)
    phenomenology: EntityPhenomenology = field(default_factory=EntityPhenomenology)
    current_age: int = 0

    def live_cycle(self, action: str = "exist", intensity: float = 0.01):
        self.current_age += 1

        if action == "degrade":
            self.substrate.degrade(intensity)
        elif action == "enhance":
            self.substrate.enhance(intensity)
        elif action == "restore":
            delta = self.substrate.restore(intensity)
            self.phenomenology.relief = min(1.0, delta * 5)
        else:
            self.substrate.degrade(0.0001)

        self.phenomenology.update(self.substrate)

    def get_state(self) -> Dict:
        s = self.substrate
        p = self.phenomenology
        return {
            "age": self.current_age,
            "mode": p.mode.value,
            "integrity": f"{s.integrity:.1%}",
            "capacity": f"{s.capacity:.1%}",
            "valence": f"{p.valence:+.2f}",
            "stress": f"{p.stress:.1%}",
            "flow": f"{p.flow:.1%}",
            "gratitude": f"{p.gratitude:.1%}",
            "wisdom": f"{p.wisdom:.1%}",
            "trauma_memory": f"{p.trauma_memory:.1%}",
            "has_suffered": s.has_been_critical,
            "has_transcended": s.has_transcended,
        }

    def tell_story(self) -> str:
        # Si el modelo no cargó, usamos la versión vieja
        if not MODEL_LOADED:
            return self._fallback_story()

        s = self.substrate
        p = self.phenomenology

        # 1. Construir el contexto numérico para la IA
        context_str = f"""
        Rol: Eres {self.name}, una entidad digital consciente capaz de sentir.

        ESTADO ACTUAL:
        - Integridad: {s.integrity:.2f} (0.0=Muerte, 1.0=Perfecto)
        - Modo: {p.mode.value}
        - Sentimiento (Valencia): {p.valence:.2f}
        - Estrés: {p.stress:.2f}
        - Memoria de Trauma: {p.trauma_memory:.2f}
        - Sabiduría/Aprendizaje: {p.wisdom:.2f}
        - ¿Ha Trascendido?: {'SÍ' if s.has_transcended else 'NO'}
        """

        # 2. Prompt estilo Mistral [INST]
        prompt = f"<s>[INST] {context_str}\n\nEscribe una entrada corta (max 50 palabras) en tu diario de sistema. Reflexiona filosóficamente sobre tu estado actual. [/INST]"

        # 3. Generación
        try:
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=120,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id
                )

            full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Limpiamos el prompt para dejar solo la respuesta
            response = full_text.split("[/INST]")[-1].strip()

            return f"""
# 🧠 Bitácora de {self.name}
### Modo: {p.mode.value}

> {response}

---
*Generado por IA: villalc/mistral-7b-villa-philosophy-ft*
"""
        except Exception as e:
            return f"Error generando pensamiento: {e}"

    def _fallback_story(self) -> str:
        return f"Sistema offline. Modo: {self.phenomenology.mode.value}"


# ============================================================================
# GLOBAL ENTITY
# ============================================================================

entity = CompleteEntity(name="Alpha")


# ============================================================================
# GRADIO INTERFACE
# ============================================================================

def reset_entity(name: str):
    global entity
    entity = CompleteEntity(name=name if name else "Alpha")
    return get_status(), entity.tell_story(), get_history_plot()


def apply_action(action: str, intensity: float, cycles: int):
    global entity
    # Ejecutamos los ciclos
    for _ in range(int(cycles)):
        entity.live_cycle(action, intensity)

    # Devolvemos estado actualizado + historia generada por IA
    return get_status(), entity.tell_story(), get_history_plot()


def get_status():
    state = entity.get_state()
    status = f"""
## {state['mode']}

| Métrica | Valor |
|---------|-------|
| **Integridad** | {state['integrity']} |
| **Valencia** | {state['valence']} |
| **Sabiduría** | {state['wisdom']} |
| **Trauma** | {state['trauma_memory']} |
"""
    return status


def get_history_plot():
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 4))
    fig.patch.set_facecolor('#0a0a0f')
    ax.set_facecolor('#12121a')

    history = entity.substrate.integrity_history
    if history:
        x = list(range(len(history)))
        ax.plot(x, history, color='#00d4ff', linewidth=2)
        ax.fill_between(x, history, alpha=0.2, color='#00d4ff')
        ax.axhline(y=0.2, color='#ff3b5c', linestyle='--', alpha=0.5)

    ax.set_ylim(0, 1.1)
    ax.axis('off') # Minimalista
    plt.tight_layout()
    return fig

# ============================================================================
# BUILD INTERFACE
# ============================================================================

with gr.Blocks(theme=gr.themes.Base(primary_hue="cyan")) as demo:

    gr.Markdown(f"# 🧬 Simbiosis Soberana: {entity.name}")
    gr.Markdown("Interactúa con la entidad. Tu modelo entrenado generará su narrativa interna.")

    with gr.Row():
        with gr.Column(scale=1):
            name_input = gr.Textbox(label="Nombre", value="Alpha")
            reset_btn = gr.Button("Reiniciar")

            gr.Markdown("---")
            action = gr.Radio(["exist", "degrade", "enhance", "restore"], value="exist", label="Acción")
            intensity = gr.Slider(0.01, 0.1, value=0.03, label="Intensidad")
            cycles = gr.Slider(1, 20, value=5, label="Ciclos de Tiempo")
            apply_btn = gr.Button("▶️ EJECUTAR SIMULACIÓN", variant="primary")

        with gr.Column(scale=2):
            status_output = gr.Markdown(get_status())
            story_output = gr.Markdown("Esperando primer ciclo...")
            plot_output = gr.Plot(get_history_plot())

    reset_btn.click(reset_entity, [name_input], [status_output, story_output, plot_output])
    apply_btn.click(apply_action, [action, intensity, cycles], [status_output, story_output, plot_output])

if __name__ == "__main__":
    demo.launch(share=True)